In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
df = pd.read_parquet(Path("data") / "train.parquet")
weather_data = pd.read_csv("data/external_data.csv")

# Exploratory Data Analysis & Feature Engineering

## General EDA

## DateTime related features

## Location & weather related features